# Inaugural Project

> **Note the following:** 
> 1. This is an example of how to structure your **inaugural project**.
> 1. Remember the general advice on structuring and commenting your code
> 1. The `inauguralproject.py` file includes a function which can be used multiple times in this notebook.

Imports and set magics:

In [8]:
import numpy as np

# autoreload modules when code is run. Otherwise, python will not see recent changes. 
%load_ext autoreload
%autoreload 2

# Import your own code
from PythonAflevering1 import HouseholdSpecializationModelClass
import matplotlib.pyplot as plt

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Question 1

**Explain how you solve the model**

In [7]:


sol_q1 = []

alpha_q1 = [0.25, 0.50, 0.75] #alpha værdierne defineres i en liste
sigma_q1 = [0.5, 1.0, 1.5] #sigma værdierne defineres i en liste

for a in alpha_q1:
    for s in sigma_q1:
        model_q1 = HouseholdSpecializationModelClass()
        model_q1.par.alpha = a #Vi sætter vores værdi for alpha ind i modellen
        model_q1.par.sigma = s #Vi sætter vores værdi for sigma ind i modellen
        opt = model_q1.solve_discrete() #Modellen løses
        HFHM1 = opt.HF/opt.HM #Forholdet mellem HF og HM genereres
        print(f"alpha = {a:.3f}, sigma = {s:.3f}, forhold = {HFHM1:.3f}") #Løsningen printes
        sol_q1.append(HFHM1)
        

c:\Users\bjorn\OneDrive - University of Copenhagen\4. semester\projects-2023-pied-piper\inauguralproject\PythonAflevering1.py:62: RuntimeWarning: divide by zero encountered in reciprocal
  H = ((1-par.alpha) * HM**((par.sigma-1)/par.sigma) + par.alpha * HF**((par.sigma-1)/par.sigma))**(par.sigma/(par.sigma-1))


alpha = 0.250, sigma = 0.500, forhold = 0.545
alpha = 0.250, sigma = 1.000, forhold = 0.308
alpha = 0.250, sigma = 1.500, forhold = 0.214
alpha = 0.500, sigma = 0.500, forhold = 1.000
alpha = 0.500, sigma = 1.000, forhold = 1.000
alpha = 0.500, sigma = 1.500, forhold = 1.000
alpha = 0.750, sigma = 0.500, forhold = 1.833
alpha = 0.750, sigma = 1.000, forhold = 3.250
alpha = 0.750, sigma = 1.500, forhold = 4.667


# Question 2

Explain your code and procedure

In [ ]:

#wF værdier sættes i en liste
var_wF_q2 = [0.8,0.9,1.0,1.1,1.2]

#Tom liste laves til værdierne
sol_wF_q2 = []
sol_HFHM_q2 = []


for w in var_wF_q2:
    model_q2 = HouseholdSpecializationModelClass()
    model_q2.par.wF = w # wF sættes ind i modellen
    opt_q2 = model_q2.solve_discrete()


    HFHM = opt_q2.HF/opt_q2.HM #HFHM forhold defineres
    sol_HFHM_q2.append(HFHM)


    WFWM = w/model_q2.par.wM # vi finder forholdet mellem wF og wM
    sol_wF_q2.append(WFWM) #Forholdet sættes i listen
    #print(f"forhold = {WFWM:.3f}") # Forholdet printes


log_sol_wF_q2 = np.log(sol_wF_q2) # Der tages log til forholdene
log_sol_HFHM_q2 = np.log(sol_HFHM_q2) 

print(log_sol_wF_q2) # De printes for at se resultater
print(log_sol_HFHM_q2)

In [ ]:
%matplotlib inline
from mpl_toolkits.mplot3d import Axes3D # Used implictely when doing 3D plots
import matplotlib.pyplot as plt # baseline module

# Predefine options for all plots
plt.rcParams.update({"axes.grid":True,"grid.color":"black","grid.alpha":"0.25","grid.linestyle":"-"})
plt.rcParams.update({'font.size': 14})

In [ ]:
# a. create the figure
fig = plt.figure()

# b. plot
ax = fig.add_subplot(1,1,1)

ax.plot(log_sol_wF_q2,log_sol_HFHM_q2)

ax.set_title('Plot 1')
ax.set_xlabel('$log(wF/wM)$')
ax.set_ylabel('$log(HF/HM)$');

# Question 3

Explain your code and procedure

In [6]:
from scipy.optimize import minimize
model_q3 = HouseholdSpecializationModelClass()

var_WF_q3 = [0.8,0.9,1.0,1.1,1.2]

list_HFHM_q3 = []
list_WFWM_q3 = []

for w in var_WF_q3 :
    model_q3 = HouseholdSpecializationModelClass()
    model_q3.par.wF = w
    opt_q3 = model_q3.solve()
    
    
    HFHM = model_q3.sol.HF/model_q3.sol.HM #HFHM forhold defineres
    list_HFHM_q3.append(HFHM)
    
    
    WFWM = w/model_q3.par.wM # vi finder forholdet mellem wF og wM
    list_WFWM_q3.append(WFWM)

log_sol_WFWM_q3 = np.log(list_WFWM_q3) # Der tages log til forholdene
log_sol_HFHM_q3 = np.log(list_HFHM_q3)

print(log_sol_WFWM_q3) # De printes for at se resultater
print(log_sol_HFHM_q3) 

[-0.22314355 -0.10536052  0.          0.09531018  0.18232156]
[ 2.25199585e-01  9.75472906e-05  2.15435980e-09 -8.68013145e-05
 -1.86626258e-01]


In [ ]:
%matplotlib inline
from mpl_toolkits.mplot3d import Axes3D # Used implictely when doing 3D plots
import matplotlib.pyplot as plt # baseline module

# Predefine options for all plots
plt.rcParams.update({"axes.grid":True,"grid.color":"black","grid.alpha":"0.25","grid.linestyle":"-"})
plt.rcParams.update({'font.size': 14})

In [ ]:
# a. create the figure
fig = plt.figure()

# b. plot
ax = fig.add_subplot(1,1,1)

ax.plot(log_sol_WFWM_q3,log_sol_HFHM_q3)

ax.set_title('Plot 1')
ax.set_xlabel('$log(wF/wM)$')
ax.set_ylabel('$log(HF/HM)$');

Opgave 4

In [9]:
from scipy import optimize


model = HouseholdSpecializationModelClass()


def solve_4(x):
    

    model.par.alpha = x[0]
    model.par.sigma = x[1]
    
    model.solve_wF_vec()
    
    model.run_regression()
    
    return ((0.4-model.sol.beta0)**2 + (-0.1-model.sol.beta1)**2)


model = HouseholdSpecializationModelClass()
bounds = [(0, 1), (0, 10)]
initial_guess = [0.5,1]
result = optimize.minimize(solve_4, initial_guess, bounds=bounds, method = "Nelder-Mead")

print(f"Optimal values: alpha = {result.x[0]:.3f}, sigma = {result.x[1]:.3f}")



AttributeError: 'NoneType' object has no attribute 'LF'

ADD CONCISE CONLUSION.